In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import os
import re
from cytoolz import concat
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def day(l):
    a = str(l)
    return a[:10]

def wordfreq(directory):
    C = Counter()
    for filename in os.listdir(directory):
        if filename.endswith('.xlsx'):
            df = pd.read_excel(os.path.join(directory, filename))
            df = df[df['LANGUAGE']=='en'].copy()
            df['bow'] = df['TEXT'].str.lower().str.replace(r'(https?://.+|[^\w#@]+|\d+)+',' ').str.split()
            df = df[df['bow'].isnull()==False].copy()
            df = df[(df['SOURCE'].str.contains('bot', case=False)==False) | (df['SOURCE'].str.contains('tweetbot', case=False)==True)]
            df = df[df['TEXT'].str.contains('HDMA')==False]
            df = df[df['TEXT'].str.contains('FALSE')==False]
            df = df[df['TEXT'].str.contains('TRUE')==False].copy()
            C += Counter(list(concat(df['bow'])))
        else:
            continue
    freq = pd.DataFrame.from_dict(C, orient='index')
    freq.columns = ['Total']
    return freq

You must EITHER run the next cell on a folder containing all anger data OR run the cell after that to import previously calculated word frequencies.
The CSV file name (for importing or exporting) will need to be changed appropriately, but this dataframe must be called freq.

In [2]:
#If overall frequency counts have not yet been generated
#If frequency counts were previously generated, skip to next cell
freq = wordfreq('INSERT PATH TO DATA DIRECTORY HERE')
freq.sort_values(by='Total',ascending=False).to_csv('INSERT FILENAME HERE.csv')

In [3]:
#Skip if frequency counts were calculated in previous step
freq = pd.read_csv('INSERT PATH TO PREVIOUSLY GENERATED FREQUENCY COUNTS HERE.csv',index_col=0)

In [4]:
#To calculate PMI for one date in one city
def PMI(filepath, date):
    df = freq
    df2 = pd.read_excel(filepath)
    df2 = df2[df2['LANGUAGE']=='en'].copy()
    df2['bow'] = df2['TEXT'].str.lower().str.replace(r'(https?://.+|[^\w#@]+|\d+)+',' ').str.split()
    df2 = df2[df2['bow'].isnull()==False].copy()
    df2 = df2[(df2['SOURCE'].str.contains('bot', case=False)==False) | (df2['SOURCE'].str.contains('tweetbot', case=False)==True)]
    df2 = df2[df2['TEXT'].str.contains('FALSE')==False].copy()
    df2 = df2[df2['TEXT'].str.contains('TRUE')==False].copy()
    df2['Day'] = df2['CREATED_AT_LOCAL'].apply(day)
    aday = df2[df2['Day']==date]
    F = pd.value_counts(list(concat(aday['bow'])))
    df[date] = F
    df['PMI'] = np.log2((df[date]*np.sum(df['Total'])) / (df['Total']*np.sum(df[date])))
    one = df[df[date] > 10].sort_values(by=['PMI'],ascending=False)
    city = df2['CITY'][0]
    one.to_csv(date + ' ' + city + ' PMI.csv')
    del df[date]
    del df['PMI']

#To calculate PMI for one date for multiple cities
def PMIdir(directory, date):
    for filename in os.listdir(directory):
        if filename.endswith('.xlsx'):
            PMI(os.path.join(directory, filename), date)
            
#To calculate PMI for multiple dates and multiple cities
def PMIdir_dates(directory, datelist):
    for filename in os.listdir(directory):
        if filename.endswith('.xlsx'):
            for date in datelist:
                PMI(os.path.join(directory, filename), date)

Generate Word Clouds From PMI Files Produced By The Functions Above

Word clouds will automatically be saved to files, two per PMI list.

In [ ]:
#For words with PMI > 1, by count. Larger size indicates more uses of the word.
def countpmiwc(filepath):
    df = pd.read_csv(filepath, index_col=0)
    df.columns = ['Total','Count','PMI']
    name = os.path.basename(filepath).split('.')[0]
    df2 = df[df['PMI']>=1].copy()
    df2['Count'] = df2['Count'].astype(float)
    v = dict(df2['Count'])
    i = WordCloud(width=1000,height=1000).generate_from_frequencies(v).to_file(name + ' Word Cloud by Count.jpg')

#For all words with PMI > 0. Larger size indicates higher PMI.
def pmiwc(filepath):
    df = pd.read_csv(filepath, index_col=0)
    df.columns = ['Total','Count','PMI']
    name = os.path.basename(filepath).split('.')[0]
    df3 = df[df['PMI']>0].copy()
    w = dict(df3['PMI'])
    j = WordCloud(width=1000,height=1000).generate_from_frequencies(w).to_file(name + ' Word Cloud.jpg')
    
def wordclouds(filepath):
    countpmiwc(filepath)
    pmiwc(filepath)
    
def wcfolder(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            wordclouds(os.path.join(directory, filename))
            continue
        else:
            continue